In [ ]:
from traitlets import List, Any
import numpy as np
import anywidget
from pathlib import Path
import solara


class TldrawFromPoints(anywidget.AnyWidget):
    points = List(List(Any())).tag(sync=True)
    _esm = Path.cwd() / "src" / "tldraw" / "static" / "points_from_numpy.js"


float_value = solara.reactive(0)


@solara.component
def Page():
    dispersion = "high"  # "high", "low", "const"

    def n(wj):
        if dispersion == "high":
            return 1 + wj * 0.1
        if dispersion == "low":
            return 1.1 - wj * 0.01
        if dispersion == "const":
            return 1

    c = 1
    num_of_waves = 121
    start_w = 1
    end_w = 7
    x = np.linspace(-2, 10, 1001)

    def g(x, t):
        u1 = 0
        for wj in np.linspace(start_w, end_w, num_of_waves):
            u1 += np.exp(1j * (wj * n(wj) / c * x - t * wj))
        return u1.real

    offset = 38
    points = np.column_stack((x * 50 + offset, g(x, float_value.value) + 150)).tolist()
    points = points[100:]
    points[0] = [offset - 40, 0]

    with solara.Row():
        solara.Button("Reset", on_click=lambda: float_value.set(0))
        solara.FloatSlider("Time t", value=float_value, min=0, max=4 * np.pi, step=0.1)
    TldrawFromPoints.element(points=points)


Page()

In [ ]:
# hosting this webapp-> https://huggingface.co/spaces/kolibril13/tldraw-solara-test